In [12]:
!pip install pandas numpy matplotlib seaborn scikit-learn tensorflow optuna



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import optuna
import joblib

In [31]:

# Cargar los datos preprocesados
X_train_scaled_imputed = np.load('X_train.npy')
X_test_scaled_imputed = np.load('X_test.npy')
y_train = np.load('y_train.npy')

# Cargar test_data y sus índices
test_data = pd.DataFrame(np.load('test_data.npy'), columns=['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity'])
test_data_index = np.load('test_data_index.npy')

# Obtener el índice real de test_data
test_data.index = test_data_index

# Cargar el objeto scaler
scaler = StandardScaler()
X_train_scaled_imputed = scaler.fit_transform(X_train_scaled_imputed)
X_test_scaled_imputed = scaler.transform(X_test_scaled_imputed)

In [32]:
# Modelo de Deep Learning
model_dl = Sequential()
model_dl.add(Dense(30, activation='relu', input_shape=(X_train_scaled_imputed.shape[1],)))
model_dl.add(Dense(1))
model_dl.compile(optimizer='adam', loss='mean_squared_error')

model_dl.fit(X_train_scaled_imputed, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
413/413 [==============================] - 1s 1ms/step - loss: 56150765568.0000 - val_loss: 56865390592.0000
Epoch 2/50
413/413 [==============================] - 0s 925us/step - loss: 56135921664.0000 - val_loss: 56842342400.0000
Epoch 3/50
413/413 [==============================] - 0s 980us/step - loss: 56106373120.0000 - val_loss: 56805396480.0000
Epoch 4/50
413/413 [==============================] - 0s 960us/step - loss: 56063217664.0000 - val_loss: 56754757632.0000
Epoch 5/50
413/413 [==============================] - 0s 930us/step - loss: 56007954432.0000 - val_loss: 56693112832.0000
Epoch 6/50
413/413 [==============================] - 0s 923us/step - loss: 55942520832.0000 - val_loss: 56621686784.0000
Epoch 7/50
413/413 [==============================] - 0s 958us/step - loss: 55867674624.0000 - val_loss: 56541065216.0000
Epoch 8/50
413/413 [==============================] - 0s 1ms/step - loss: 55784124416.0000 - val_loss: 56451645440.0000
Epoch 9/50
413/413 [========

In [33]:
# Modelo de Machine Learning
model_ml = RandomForestRegressor(n_estimators=100, random_state=42)
model_ml.fit(X_train_scaled_imputed, y_train)

RandomForestRegressor(random_state=42)

In [34]:
# Evaluación del modelo de Deep Learning
y_test = test_data["median_house_value"].copy()
X_test_scaled_imputed = scaler.transform(X_test_scaled_imputed)  # Corregir esta línea
y_pred_dl_imputed = model_dl.predict(X_test_scaled_imputed)
mse_dl_imputed = mean_squared_error(y_test, y_pred_dl_imputed)
r2_dl_imputed = r2_score(y_test, y_pred_dl_imputed)
print(f"Deep Learning con Imputación - Mean Squared Error: {mse_dl_imputed}, R^2 Score: {r2_dl_imputed}")


129/129 [==============================] - 1s 701us/step
Deep Learning con Imputación - Mean Squared Error: 45762656040.59284, R^2 Score: -2.4922422541720435


In [35]:
# Evaluación del modelo de Machine Learning
y_pred_ml_imputed = model_ml.predict(X_test_scaled_imputed)
mse_ml_imputed = mean_squared_error(y_test, y_pred_ml_imputed)
r2_ml_imputed = r2_score(y_test, y_pred_ml_imputed)
print(f"Machine Learning con Imputación - Mean Squared Error: {mse_ml_imputed}, R^2 Score: {r2_ml_imputed}")



Machine Learning con Imputación - Mean Squared Error: 2474438338.6272635, R^2 Score: 0.8111705292229482


In [ ]:
# Ajuste de Hiperparámetros (utilizando Optuna)
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train_scaled_imputed, y_train)
    y_pred = model.predict(X_test_scaled_imputed)
    return mean_squared_error(y_test, y_pred)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

best_params = study.best_params
best_model = RandomForestRegressor(n_estimators=best_params['n_estimators'],
                                    max_depth=best_params['max_depth'], random_state=42)
best_model.fit(X_train_scaled_imputed, y_train)

y_pred_best = best_model.predict(X_test_scaled_imputed)
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)
print(f"Best Model - Mean Squared Error: {mse_best}, R^2 Score: {r2_best}")


[I 2024-01-13 02:54:35,162] A new study created in memory with name: no-name-490ae96b-bdfa-4881-92fc-ed2061761f9b
[I 2024-01-13 02:54:39,816] Trial 0 finished with value: 3595974529.9973273 and parameters: {'n_estimators': 48, 'max_depth': 8}. Best is trial 0 with value: 3595974529.9973273.
[I 2024-01-13 02:55:03,064] Trial 1 finished with value: 2488194373.625022 and parameters: {'n_estimators': 89, 'max_depth': 23}. Best is trial 1 with value: 2488194373.625022.
[I 2024-01-13 02:55:53,122] Trial 2 finished with value: 2478521373.329473 and parameters: {'n_estimators': 190, 'max_depth': 27}. Best is trial 2 with value: 2478521373.329473.
[I 2024-01-13 02:56:29,860] Trial 3 finished with value: 2548863645.472299 and parameters: {'n_estimators': 195, 'max_depth': 14}. Best is trial 2 with value: 2478521373.329473.
[I 2024-01-13 02:56:44,020] Trial 4 finished with value: 2503910022.5438714 and parameters: {'n_estimators': 56, 'max_depth': 21}. Best is trial 2 with value: 2478521373.32947

In [ ]:
# Ruta donde se guardarán los modelos
ruta_modelos = 'modelos/'

# Guardar el modelo de Deep Learning
model_dl.save(f'{ruta_modelos}modelo_dl')

# Guardar el modelo de Machine Learning
ruta_modelo_ml = f'{ruta_modelos}modelo_ml.joblib'
joblib.dump(model_ml, ruta_modelo_ml)

# Guardar el mejor modelo
ruta_mejor_modelo = f'{ruta_modelos}mejor_modelo.joblib'
joblib.dump(best_model, ruta_mejor_modelo)

# Puedes imprimir un mensaje indicando la ubicación de los modelos guardados
print(f"Modelos guardados en: {ruta_modelos}")